<a href="https://colab.research.google.com/github/chettkulkarni/MachineLearning/blob/master/weatherDataLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/open?id=1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='


downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Fire.csv')

1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9


In [0]:
#Api to fetch the records

# response = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2017-01-01&enddate=2017-01-31&locationid=FIPS:23&format=csv", headers={'token': 'RGFmBkMzqCUgFXbiQWeqalQIAAoBMhTQ'})
# response = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories", headers={'token': 'RGFmBkMzqCUgFXbiQWeqalQIAAoBMhTQ'})

# response.content

In [0]:
# responseObject=response.json()

In [0]:
# df=pd.DataFrame.from_dict(responseObject['results'])

In [42]:
# df
files = [str(x)+'.csv.gz' for x in range(1993,1994)]
files

['1993.csv.gz']

In [43]:
#fetching data from ftp server for years from 1985 to 2000

from ftplib import FTP
from datetime import datetime

start = datetime.now()
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()
ftp.cwd("pub/data/ghcn/daily/by_year/")


# Get All Files
files = [str(x)+'.csv.gz' for x in range(1993,1994)]

# Print out the files
for file in files:
	print("Downloading..." + file)
	ftp.retrbinary("RETR " + file ,open(file, 'wb').write)

ftp.close()

end = datetime.now()
diff = end - start
print('All files downloaded for ' + str(diff.seconds) + 's')

Downloading...1993.csv.gz
All files downloaded for 24s


In [0]:
# df = pd.read_csv('1987.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)

In [0]:
# df

In [0]:
#getting ready with sparkx instance

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!pip install pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkFiles

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
#generating spark contect and sessions to consume our datasets

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("gfcj") \
        .getOrCreate()

In [0]:
#adding all the files to spark context to spread them into rdd and perform faster exexution on huge dataset

In [0]:
sc =SparkContext.getOrCreate()

for file in files:
  sc.addFile(file)

In [0]:
sqlContext = SQLContext(sc)
sqlContext
# df = sqlContext.read.csv(SparkFiles.get(files), header=True, inferSchema= True)
# df.printSchema()

spark.read.format("csv").option("header", "true").load("*.csv.gz")
df=spark.read.option("header", "false").csv("*.csv.gz")

In [46]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [47]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|CA002303986|19930101|TMAX|-140|null|null|  C|null|
|CA002303986|19930101|TMIN|-280|null|null|  C|null|
|CA002303986|19930101|PRCP|   0|null|null|  C|null|
|CA002303986|19930101|SNOW|   0|null|null|  C|null|
|CA002303986|19930101|SNWD| 200|null|null|  C|null|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



In [0]:
sc.addFile('ghcnd-stations.csv')

In [0]:
df2=spark.read.option("header", "true").csv('ghcnd-stations.csv')

In [50]:
df2

DataFrame[id: string, lat: string, long: string, _c3: string, _c4: string, _c5: string]

In [51]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|CA002303986|19930101|TMAX|-140|null|null|  C|null|
|CA002303986|19930101|TMIN|-280|null|null|  C|null|
|CA002303986|19930101|PRCP|   0|null|null|  C|null|
|CA002303986|19930101|SNOW|   0|null|null|  C|null|
|CA002303986|19930101|SNWD| 200|null|null|  C|null|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



In [52]:
df2.show(5)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
|AE000041196| 25.333|  55.517|null|null|null|
|AEM00041194| 25.255|  55.364|null|null|null|
|AEM00041217| 24.433|  54.651|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 5 rows



In [0]:
data=df.selectExpr('_c0 as id','_c1 as date','_c2 as type','_c3 as value')

In [54]:
data.show()

+-----------+--------+----+-----+
|         id|    date|type|value|
+-----------+--------+----+-----+
|CA002303986|19930101|TMAX| -140|
|CA002303986|19930101|TMIN| -280|
|CA002303986|19930101|PRCP|    0|
|CA002303986|19930101|SNOW|    0|
|CA002303986|19930101|SNWD|  200|
|ASN00037003|19930101|PRCP|    0|
|USC00242347|19930101|TMAX| -183|
|USC00242347|19930101|TMIN| -289|
|USC00242347|19930101|TOBS| -189|
|USC00242347|19930101|PRCP|    0|
|USC00242347|19930101|SNOW|    0|
|NOE00133566|19930101|TMAX|   85|
|NOE00133566|19930101|TMIN|   41|
|NOE00133566|19930101|PRCP|   30|
|NOE00133566|19930101|SNWD|    0|
|ASN00028020|19930101|PRCP|  120|
|USS0018D08S|19930101|TMAX|  -53|
|USS0018D08S|19930101|TMIN| -112|
|USS0018D08S|19930101|TOBS| -106|
|USS0018D08S|19930101|PRCP|   76|
+-----------+--------+----+-----+
only showing top 20 rows



In [55]:
df2.show(2)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 2 rows



In [0]:
final_df = data.join(df2, on=['id'], how='inner')

In [0]:
final_df=final_df.selectExpr('id','lat as LATITUDE','long as LONGITUDE','date','type','value')

In [0]:
final_df.registerTempTable("temperature")

In [0]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [0]:
# final_df.shape()

In [0]:
# df.shape()

In [26]:
fire=pd.read_csv('Fire.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=fire

df['disc_clean_date'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Convert containment date from julian to standard date
df['cont_clean_date'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Day of month string
df['discovery_month'] = df['disc_clean_date'].dt.strftime('%b')
#Returns the weekday string
df['discovery_weekday'] = df['disc_clean_date'].dt.strftime('%a')
#Merge discovery date and time 


In [0]:
df_1993=df[df.FIRE_YEAR==1993]

In [0]:
df_1993=df_1993[['LATITUDE','LONGITUDE','cont_clean_date']]

In [90]:
df_1993.round()

,LATITUDE,LONGITUDE,cont_clean_date
53303,46.0,-113.0,1993-05-28
53304,46.0,-113.0,1993-08-14
53305,45.0,-112.0,1993-08-20
53306,45.0,-112.0,1993-10-26
53307,45.0,-111.0,1993-06-28
...,...,...,...
1635058,31.0,-99.0,NaT
1860063,46.0,-121.0,1993-08-21
1861394,21.0,-158.0,NaT
1861400,21.0,-158.0,NaT


In [0]:
df_1993=df_1993.dropna()

In [92]:
df_1993

,LATITUDE,LONGITUDE,cont_clean_date
53303,45.913333,-113.008333,1993-05-28
53304,45.503333,-113.008333,1993-08-14
53305,45.433333,-112.041667,1993-08-20
53306,44.986667,-111.900000,1993-10-26
53307,45.335000,-111.433333,1993-06-28
...,...,...,...
1421217,32.766900,-83.466200,1993-10-18
1421218,34.875800,-85.136500,1993-09-12
1421219,34.378500,-84.323200,1993-09-13
1421221,33.641000,-83.097700,1993-07-15


In [93]:
type(df_1993['cont_clean_date'])

pandas.core.series.Series

In [0]:
df_1993['cont_clean_date']=df_1993['cont_clean_date'].astype(str)

In [0]:
df_1993['date']=df_1993['cont_clean_date'].replace('-', '', regex=True).astype(int)

In [96]:
df_1993

,LATITUDE,LONGITUDE,cont_clean_date,date
53303,45.913333,-113.008333,1993-05-28,19930528
53304,45.503333,-113.008333,1993-08-14,19930814
53305,45.433333,-112.041667,1993-08-20,19930820
53306,44.986667,-111.900000,1993-10-26,19931026
53307,45.335000,-111.433333,1993-06-28,19930628
...,...,...,...,...
1421217,32.766900,-83.466200,1993-10-18,19931018
1421218,34.875800,-85.136500,1993-09-12,19930912
1421219,34.378500,-84.323200,1993-09-13,19930913
1421221,33.641000,-83.097700,1993-07-15,19930715


In [0]:
df_1993=df_1993[['LATITUDE','LONGITUDE','date']]

In [0]:
df_1993.to_csv(r'fire_1993.csv')

In [0]:
sc.addFile('fire_1993.csv')

In [0]:
fire=spark.read.option("header", "true").csv('fire_1993.csv')

In [101]:
fire.show(5)

+-----+-----------+-------------+--------+
|  _c0|   LATITUDE|    LONGITUDE|    date|
+-----+-----------+-------------+--------+
|53303|45.91333333|-113.00833333|19930528|
|53304|45.50333333|-113.00833333|19930814|
|53305|45.43333333|-112.04166667|19930820|
|53306|44.98666667|       -111.9|19931026|
|53307|     45.335|-111.43333333|19930628|
+-----+-----------+-------------+--------+
only showing top 5 rows



In [0]:
complete = fire.join(final_df, on=['LATITUDE','LONGITUDE','date'], how='inner')

In [110]:
complete.show()

+--------+---------+--------+-----+-----------+----+-----+
|LATITUDE|LONGITUDE|    date|  _c0|         id|type|value|
+--------+---------+--------+-----+-----------+----+-----+
|  35.145| -111.675|19930731|54656|USR0000AFLG|TMAX|  322|
|  35.145| -111.675|19930731|54656|USR0000AFLG|TMIN|  117|
|  35.145| -111.675|19930731|54656|USR0000AFLG|TAVG|  223|
+--------+---------+--------+-----+-----------+----+-----+

